In [ ]:
## path to the datasets
datadir='./Datasets/'

## required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import igraph as ig
import partition_igraph
from collections import Counter
from sklearn.metrics import adjusted_mutual_info_score as AMI
import random


## 2.4 Bonus module - Community-based anomaly detection

### New dataset -- American College Football Graph

This is a nice, small graph for illustrating anomaly detection methods.

The graph consists of 115 US college football teams (nodes) playing games (edges).

Teams are part of 12 conferences (the 'communities'):
*   0 = Atlantic Coast
*   1 = Big East
*   2 = Big Ten
*   3 = Big Twelve
*   4 = Conference USA
*   5 = Independents
*   6 = Mid-American
*   7 = Mountain West
*   8 = Pacific Ten
*   9 = Southeastern
*  10 = Sun Belt
*  11 = Western Athletic

14 teams out of 115 appear as "anomalies", namely:
- the 5 teams in #5 conference (Independent) play teams in other conferences 
- the 7 teams in #10 conference (Sun Belt) are broken in 2 clumps 
- 2 teams from #11 conference play mainly with #10 conference

[REF]: "Community structure in social and biological networks", M. Girvan and M. E. J. Newman
PNAS June 11, 2002 99 (12) 7821-7826; https://doi.org/10.1073/pnas.122653799


In [ ]:
cfg = ig.Graph.Read_Ncol(datadir+'Football/football.edgelist',directed=False)
c = np.loadtxt(datadir+'Football/football.community',dtype='uint16',usecols=(0))
cfg.vs['community'] = [c[int(x['name'])] for x in cfg.vs]

Let's look at the College Football Graph and show communities in different colors, and anomalies as squares

In [ ]:
cfg.vs['shape'] = 'circle'
cfg.vs['anomaly'] = 0
pal = ig.ClusterColoringPalette(n=max(cfg.vs['community'])+1) 
cfg.vs['color'] = [pal.get(int(i)) for i in cfg.vs['community']]
for v in cfg.vs:
    if v['community'] in [5,10] or v['name'] in ['28','58']:
        v['shape']='square'
        v['anomaly']=1
ly = cfg.layout_fruchterman_reingold()
ig.plot(cfg, layout=ly, bbox=(0,0,500,300), vertex_size=6, edge_color='lightgray')


In [ ]:
## cluster with ECG
ec = cfg.community_ecg()
cfg.vs['ecg'] = ec.membership
print('number of communities found:',np.max(cfg.vs['ecg'])+1)

In [ ]:
## AMI (<=1) in a measure of the cluster quality given ground-truth
AMI(cfg.vs['community'],cfg.vs['ecg'])

Compute and compare the participation coefficients; a **high** value indicative of **outlier**

In [ ]:
## measures the homogeneity of a list
def partCoef(l):
    s = sum(l)
    pc = 1-sum([i**2/s**2 for i in l]) 
    return pc

## for each node, list the clusters of its neighbours and compute the participation coefficient
for v in cfg.vs:
    l = list(Counter([cfg.vs[x]['ecg'] for x in cfg.neighbors(v)]).values()) ## neighbour's communities    
    v['pc'] = partCoef(l)

## boxplot - outliers vs "regular" nodes
plt.rcParams['font.size'] = '14'
X = [v['pc'] for v in cfg.vs if v['anomaly']==0]
Y = [v['pc'] for v in cfg.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Outlier'],sym='.',whis=(0,100), widths=.5)
plt.ylabel('Participation coefficient',fontsize=14);

With ECG, node strongly in a community should have **high** "ecg weights" on edges linking its neighbours

In [ ]:
## collect ECG weights
cfg.es['ecg_weight'] = ec.W

## average those for each node
for v in cfg.vs:
    v['ecg'] = np.mean([cfg.es[e]['ecg_weight'] for e in cfg.incident(v)])

## boxplot - outliers vs "regular" nodes
X = [v['ecg'] for v in cfg.vs if v['anomaly']==0]
Y = [v['ecg'] for v in cfg.vs if v['anomaly']==1]
plt.boxplot([X,Y],labels=['Regular','Outlier'],sym='.',whis=(0,100), widths=.5)
plt.ylabel('ECG weights',fontsize=14);
